In [1]:
__depends__ = []
__dest__ = [
    "../../../data/database-results/phase-1/timestamp.txt",
    "../../../data/database-observations/timestamp.txt",
]

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

# Database generation

In this notebook we process the data into a database we can later query to make plots/do analysis etc.

## Imports

In [3]:
import logging
import os
import os.path
import re
from datetime import datetime
from pathlib import Path
from pprint import pprint
from time import sleep
from distutils.util import strtobool

import pandas as pd
import pyam
import tqdm
from scmdata import ScmDataFrame, df_append

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
TEST_RUN = strtobool(os.getenv("CI", "False")) or False
TEST_RUN

False

<IPython.core.display.Javascript object>

In [5]:
logger = logging.getLogger()

<IPython.core.display.Javascript object>

## Constants

In [6]:
OUTPUT_DATABASE_PATH = os.path.join(
    "..", "..", "..", "data", "database-results", "phase-1"
)

OBS_DATABASE_PATH = os.path.join("..", "..", "..", "data", "database-observations")

<IPython.core.display.Javascript object>

In [7]:
if not os.path.isdir(OUTPUT_DATABASE_PATH):
    os.mkdir(OUTPUT_DATABASE_PATH)

if not os.path.isdir(OBS_DATABASE_PATH):
    os.mkdir(OBS_DATABASE_PATH)

<IPython.core.display.Javascript object>

## Miscellaneous functions

TODO: put these into some sort of `utils` file

In [8]:
def strip_quantile(inv):
    if inv.endswith("mean"):
        return "|".join(inv.split("|")[:-1])

    if inv.endswith("stddev"):
        return "|".join(inv.split("|")[:-1])

    if "quantile" in inv:
        if re.match(".*\|([1-9]\d?|100|0|[1-9]\d*\.\d)th quantile$", inv) is None:
            print("Bad formatting: {}".format(inv))

        return "|".join(inv.split("|")[:-1])

    return inv


def check_all_variables_and_units_as_in_protocol(df_to_check, protocol_variables):
    checker_df = df_to_check.filter(variable="*Other*", keep=False)[
        ["variable", "unit"]
    ]
    checker_df["unit"] = checker_df["unit"].apply(
        lambda x: x.replace("dimensionless", "Dimensionless")
        if isinstance(x, str)
        else x
    )

    def strip_quantile(inv):
        if any([inv.endswith(suf) for suf in ["quantile", "mean", "stddev"]]):
            return "|".join(inv.split("|")[:-1])

        return inv

    checker_df["variable"] = checker_df["variable"].apply(strip_quantile)
    merged_df = checker_df.merge(protocol_variables[["variable", "unit"]])
    try:
        assert len(merged_df) == len(checker_df)
    except AssertionError:
        pprint(set(checker_df["variable"]) - set(protocol_variables["variable"]))
        pprint(set(checker_df["unit"]) - set(protocol_variables["unit"]))
        raise

<IPython.core.display.Javascript object>

In [9]:
def check_all_scenarios_as_in_protocol(df_to_check, protocol_scenarios):
    checker_df = df_to_check["scenario"].to_frame()
    merged_df = checker_df.merge(protocol_scenarios[["scenario"]])
    assert len(merged_df) == len(checker_df), set(checker_df["scenario"]) - set(
        merged_df["scenario"]
    )

<IPython.core.display.Javascript object>

In [10]:
def unify_units(in_df, protocol_variables, exc_info=False):
    out_df = in_df.copy()
    for variable in tqdm.tqdm_notebook(out_df["variable"].unique()):
        if variable.startswith("Radiative Forcing|Anthropogenic|Albedo Change"):
            target_unit = protocol_variables[
                protocol_variables["variable"]
                == "Radiative Forcing|Anthropogenic|Albedo Change"
            ]["unit"].iloc[0]

        elif variable.startswith(
            "Effective Radiative Forcing|Anthropogenic|Albedo Change"
        ):
            target_unit = protocol_variables[
                protocol_variables["variable"]
                == "Effective Radiative Forcing|Anthropogenic|Albedo Change"
            ]["unit"].iloc[0]

        elif variable.startswith("Carbon Pool"):
            target_unit = protocol_variables[
                protocol_variables["variable"] == "Carbon Pool|Atmosphere"
            ]["unit"].iloc[0]

        elif "Other" in variable:
            target_unit = protocol_variables[
                protocol_variables["variable"]
                == "{}".format(variable.split("|Other")[0])
            ]["unit"].iloc[0]

        elif any([variable.endswith(suf) for suf in ["quantile", "mean", "stddev"]]):
            try:
                target_unit = protocol_variables[
                    protocol_variables["variable"] == "|".join(variable.split("|")[:-1])
                ]["unit"].iloc[0]
            except:
                logger.exception(
                    f"Failed to find unit for {variable}", exc_info=exc_info
                )
                continue
        else:
            try:
                target_unit = protocol_variables[
                    protocol_variables["variable"] == variable
                ]["unit"].iloc[0]
            except:
                logger.exception(
                    f"Failed to find unit for {variable}", exc_info=exc_info
                )
                continue

        try:
            if "CH4" in target_unit:
                out_df = out_df.convert_unit(
                    target_unit, variable=variable, context="CH4_conversions"
                )
                continue

            if "NOx" in target_unit:
                out_df = out_df.convert_unit(
                    target_unit, variable=variable, context="NOx_conversions"
                )
                continue

            if target_unit == "Dimensionless":
                target_unit = "dimensionless"

            out_df = out_df.convert_unit(target_unit, variable=variable)
        except:
            current_unit = out_df.filter(variable=variable)["unit"].unique()
            logger.exception(
                f"Failed for {variable} with target unit: {target_unit} and current_unit: {current_unit}",
                exc_info=exc_info,
            )

    out_df = out_df.timeseries().reset_index()
    out_df["unit_context"] = out_df["unit_context"].fillna("not_required")
    return ScmDataFrame(out_df)

<IPython.core.display.Javascript object>

In [11]:
def aggregate_variable(db_in, v_to_agg):
    v_to_agg_df = (
        db_in.filter(variable=v_to_agg, keep=False)
        .filter(
            variable="{}|*".format(v_to_agg),
            level=0,  # make sure we don't pick up e.g. HFC23|50th Percentile by accident
        )
        .timeseries()
    )
    group_idx = list(set(v_to_agg_df.index.names) - {"variable"})
    v_to_agg_df = v_to_agg_df.groupby(group_idx).sum().reset_index()
    v_to_agg_df["variable"] = v_to_agg

    db_out = db_in.append(v_to_agg_df)

    return db_out

<IPython.core.display.Javascript object>

In [12]:
def prep_str_for_filename(ins):
    return (
        ins.replace("_", "-")
        .replace("|", "-")
        .replace(" ", "-")
        .replace("(", "")
        .replace(")", "")
        .lower()
    )


def get_filename(scmdf, leader):
    climatemodel = prep_str_for_filename(
        scmdf.get_unique_meta("climatemodel", no_duplicates=True)
    )
    variable = prep_str_for_filename(
        scmdf.get_unique_meta("variable", no_duplicates=True)
    )
    region = prep_str_for_filename(scmdf.get_unique_meta("region", no_duplicates=True))

    return "{}_{}_{}_{}.csv".format(leader, climatemodel, region, variable)

<IPython.core.display.Javascript object>

In [13]:
def convert_scmdf_to_pyamdf_year_only(iscmdf):
    out = iscmdf.timeseries()
    out.columns = out.columns.map(lambda x: x.year)

    return pyam.IamDataFrame(out)

<IPython.core.display.Javascript object>

In [14]:
def save_into_database(db, db_path, filename_leader):
    for cm in tqdm.tqdm_notebook(
        db["climatemodel"].unique(), leave=False, desc="Climate models"
    ):
        db_cm = db.filter(climatemodel=cm)
        for r in tqdm.tqdm_notebook(
            db_cm["region"].unique(), leave=False, desc="Regions"
        ):
            db_cm_r = db_cm.filter(region=r)
            for v in tqdm.tqdm_notebook(
                db_cm_r["variable"].unique(), leave=False, desc="Variables"
            ):
                db_cm_r_v = ScmDataFrame(db_cm_r.filter(variable=v))
                filename = get_filename(db_cm_r_v, leader=filename_leader)
                outfile = os.path.join(db_path, filename)

                convert_scmdf_to_pyamdf_year_only(db_cm_r_v).to_csv(outfile)
                logger.debug("saved file to {}".format(outfile))

    with open(os.path.join(db_path, "timestamp.txt"), "w") as fh:
        fh.write("database written at: ")
        fh.write(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        fh.write("\n")

<IPython.core.display.Javascript object>

## Protocol

In [15]:
SCENARIO_PROTOCOL = os.path.join(
    "..", "..", "..", "data", "protocol", "rcmip-emissions-annual-means.csv"
)

<IPython.core.display.Javascript object>

In [16]:
protocol_db = ScmDataFrame(SCENARIO_PROTOCOL)
protocol_db.head()

time                                                                               1750-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1751-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1752-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1753-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1754-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1755-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable

<IPython.core.display.Javascript object>

In [17]:
protocol_db["scenario"].unique()

array(['rcp60', 'ssp370', 'ssp370-lowNTCF-aerchemmip',
       'ssp370-lowNTCF-gidden', 'historical', 'ssp434', 'ssp460', 'rcp26',
       'ssp119', 'ssp126', 'rcp85', 'ssp245', 'rcp45', 'ssp534-over',
       'ssp585', 'esm-bell-1000PgC', 'esm-bell-2000PgC',
       'esm-bell-750PgC', 'esm-pi-CO2pulse', 'esm-pi-cdr-pulse',
       'esm-piControl', 'historical-cmip5'], dtype=object)

<IPython.core.display.Javascript object>

In [18]:
DATA_PROTOCOL = os.path.join(
    "..",
    "..",
    "..",
    "data",
    "submission-template",
    "rcmip-data-submission-template.xlsx",
)

<IPython.core.display.Javascript object>

In [19]:
protocol_variables = pd.read_excel(DATA_PROTOCOL, sheet_name="variable_definitions")
protocol_variables.columns = protocol_variables.columns.str.lower()
protocol_variables.head()

,idx,category,variable,unit,definition,tier
0,1,Atmospheric Concentrations,Atmospheric Concentrations|CH4,ppb,atmospheric concentrations of CH4,1
1,2,Atmospheric Concentrations,Atmospheric Concentrations|CO2,ppm,atmospheric concentrations of CO2,1
2,3,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases,ppm,equivalent species atmospheric concentrations ...,3
3,4,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases|HFC,ppm,equivalent species atmospheric concentrations ...,3
4,5,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases|HFC|HFC125,ppt,atmospheric concentrations of HFC125,2


<IPython.core.display.Javascript object>

In [20]:
protocol_scenarios = pd.read_excel(
    DATA_PROTOCOL, sheet_name="scenario_info", skip_rows=2
)
protocol_scenarios.columns = protocol_scenarios.columns.str.lower()
protocol_scenarios.head()

,unnamed: 0,scenario,description of scenario,detailed scenario description,priority
0,Instructions/Details,Scenario identifier used in the CMIP6 modeling...,Brief description of scenario,Documentation on es-doc + some other clarifyin...,"Top priority is Tier 1, additional runs are Ti..."
1,Answers,# Scenario ID,# Scenario Description,#Scenario Specification,# Tier in RCMP
2,NaN,piControl,pre-industrial control simulation (i.e. consta...,groups should run as long as is sensible/they ...,1
3,NaN,esm-piControl,pre-industrial control simulation with zero an...,should be identical to piControl except CO2 is...,1
4,NaN,esm-piControl-allGHG,pre-industrial control simulation with zero an...,should be identical to piControl except all GH...,2


<IPython.core.display.Javascript object>

## Observations

### tas observations

These come from Chris Smith (personal email). TODO: get details from Chris about what these are.

In [21]:
TAS_OBS_PATH = os.path.join(
    "..", "..", "..", "data", "priestley-centre", "observations", "tas_obs.txt"
)

<IPython.core.display.Javascript object>

In [22]:
tas_obs_df = pd.read_csv(TAS_OBS_PATH, header=None, delim_whitespace=True)
tas_obs_df.columns = ["year", "value"]
tas_obs_df["model"] = "unspecified"
tas_obs_df["climatemodel"] = "Observations (Priestley Centre)"
tas_obs_df["scenario"] = "historical"
tas_obs_df["variable"] = "Surface Air Temperature Change"
tas_obs_df["unit"] = "K"
tas_obs_df["region"] = "World"
tas_obs_df = pyam.IamDataFrame(tas_obs_df)
tas_obs_df.head()

,model,scenario,region,variable,unit,year,climatemodel,value
0,unspecified,historical,World,Surface Air Temperature Change,K,1850.0,Observations (Priestley Centre),-0.042412
1,unspecified,historical,World,Surface Air Temperature Change,K,1851.0,Observations (Priestley Centre),0.123588
2,unspecified,historical,World,Surface Air Temperature Change,K,1852.0,Observations (Priestley Centre),0.085588
3,unspecified,historical,World,Surface Air Temperature Change,K,1853.0,Observations (Priestley Centre),0.025588
4,unspecified,historical,World,Surface Air Temperature Change,K,1854.0,Observations (Priestley Centre),0.086588


<IPython.core.display.Javascript object>

In [23]:
db_obs = pyam.concat([tas_obs_df])
db_obs.head()

,model,scenario,region,variable,unit,year,climatemodel,value
0,unspecified,historical,World,Surface Air Temperature Change,K,1850.0,Observations (Priestley Centre),-0.042412
1,unspecified,historical,World,Surface Air Temperature Change,K,1851.0,Observations (Priestley Centre),0.123588
2,unspecified,historical,World,Surface Air Temperature Change,K,1852.0,Observations (Priestley Centre),0.085588
3,unspecified,historical,World,Surface Air Temperature Change,K,1853.0,Observations (Priestley Centre),0.025588
4,unspecified,historical,World,Surface Air Temperature Change,K,1854.0,Observations (Priestley Centre),0.086588


<IPython.core.display.Javascript object>

In [24]:
save_into_database(db_obs, OBS_DATABASE_PATH, "rcmip-observations")

<IPython.core.display.Javascript object>

## Model output

In [25]:
RESULTS_PATH = os.path.join("..", "..", "..", "data", "results", "phase-1")

<IPython.core.display.Javascript object>

In [26]:
results_files = list(Path(RESULTS_PATH).rglob("*.csv")) + list(
    Path(RESULTS_PATH).rglob("*.xlsx")
)
print(len(results_files))
sorted(results_files)

454


[PosixPath('../../../data/results/phase-1/acc2/rcmip_phase-1_acc2_v1-0-0.xlsx'),
 PosixPath('../../../data/results/phase-1/acc2/rcmip_phase-1_acc2_v1-0-1.xlsx'),
 PosixPath('../../../data/results/phase-1/acc2/rcmip_phase-1_acc2_v2-0-0.xlsx'),
 PosixPath('../../../data/results/phase-1/acc2/rcmip_phase-1_acc2_v2-0-1.xlsx'),
 PosixPath('../../../data/results/phase-1/ar5ir/ar5ir-phase-1-results-v1-0-0.csv'),
 PosixPath('../../../data/results/phase-1/ar5ir/ar5ir-phase-1-results-v1-0-1.csv'),
 PosixPath('../../../data/results/phase-1/ar5ir/ar5ir-phase-1-results-v2-0-0.csv'),
 PosixPath('../../../data/results/phase-1/ar5ir/rcmip-data-submission-template-ar5ir.xlsx'),
 PosixPath('../../../data/results/phase-1/cicero-scm/RCMIP_metascenario_Cicero-SCM-ECS3-v1-0-0.csv'),
 PosixPath('../../../data/results/phase-1/cicero-scm/RCMIP_metascenario_Cicero-SCM-ECS3-v1-0-1.csv'),
 PosixPath('../../../data/results/phase-1/cicero-scm/RCMIP_metascenario_Cicero-SCM-ECS3-v2-0-0.csv'),
 PosixPath('../../../data

<IPython.core.display.Javascript object>

In [27]:
model_of_interest = [
#    ".*acc2.*v2-0-1.*",
    ".*rcmip_phase-1_cicero-scm.*v5-0-0.*",
#    ".*escimo.*v2-0-1.*",
    ".*fair-1.5-default.*",
#    ".*rcmip_phase-1_gir.*",
#    ".*greb.*v2-0-0.*",
#    ".*hector.*v2-0-0.*",
#    ".*MAGICC7.1.0aX-rcmip-phase-1.*",
    ".*rcmip_phase-1_magicc7.1.0.beta.*"
#    ".*MAGICC7.1.0aX.*",
#    ".*mce.*v2-0-1.*",
    ".*oscar.*",
#    ".*wasp.*v1-0-1.*",
]
if TEST_RUN:
    model_of_interest = [
        ".*escimo-phase-1-v2-0-1.*",
        ".*greb.*",
        ".*rcmip_phase-1_cicero-scm.*v5-0-0.*",
    ]

results_files = [
    str(p)
    for p in results_files
    if any([bool(re.match(m, str(p))) for m in model_of_interest]) and "$" not in str(p)
]
print(len(results_files))
sorted(results_files)

152


['../../../data/results/phase-1/cicero-scm/rcmip_phase-1_cicero-scm-ecs3_v5-0-0.csv',
 '../../../data/results/phase-1/cicero-scm/rcmip_phase-1_cicero-scm_v5-0-0.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-4xext_v1-0-0.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-4xext_v1-0-1.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-bgc_v1-0-0.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-bgc_v1-0-1.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-cdr_v1-0-0.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-cdr_v1-0-1.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-rad_v1-0-0.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-rad_v1-0-1.csv',
 '../../../data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2_v1-0-0.cs

<IPython.core.display.Javascript object>

In [28]:
db = []
for rf in tqdm.tqdm_notebook(results_files):
    if rf.endswith(".csv"):
        loaded = ScmDataFrame(rf)
    else:
        loaded = ScmDataFrame(rf, sheet_name="your_data")
    db.append(loaded)

db = df_append(db).timeseries().reset_index()
db["unit"] = db["unit"].apply(
    lambda x: x.replace("Dimensionless", "dimensionless") if isinstance(x, str) else x
)
db = ScmDataFrame(db)
db.head()

/home/sarambl/miniconda3/envs/rcmip_smb/lib/python3.7/site-packages/scmdata/dataframe.py:1527: UserWarning: Duplicate time points detected, the output will be the average of the duplicates. Set `duplicate_msg='return'` to examine the joint timeseries (the duplicates can be found by looking at `res[res.index.duplicated(keep=False)].sort_index()`. Set `duplicate_msg=False` to silence this message.
  warnings.warn(warn_msg)


time                                                                                       1750-01-01 00:00:00  \
model       scenario region variable                       unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2          dimensionless FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CH4 ppb           Cicero-SCM                    808.249   
                                                                         Cicero-SCM-ECS3               808.249   
                                                                         FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CO2 ppm           Cicero-SCM                    284.317   

time                                                                                       1751-01-01 00:00:00  \
model       scenario region variable                       unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2          dimensionless FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CH4 ppb           Cicero-SCM                    808.249   
                                                                         Cicero-SCM-ECS3               808.249   
                                                                         FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CO2 ppm           Cicero-SCM                    284.317   

time                                                                                       1752-01-01 00:00:00  \
model       scenario region variable                       unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2          dimensionless FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CH4 ppb           Cicero-SCM                    808.249   
                                                                         Cicero-SCM-ECS3               808.249   
                                                                         FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CO2 ppm           Cicero-SCM                    284.317   

time                                                                                       1753-01-01 00:00:00  \
model       scenario region variable                       unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2          dimensionless FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CH4 ppb           Cicero-SCM                    808.249   
                                                                         Cicero-SCM-ECS3               808.249   
                                                                         FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CO2 ppm           Cicero-SCM                    284.317   

time                                                                                       1754-01-01 00:00:00  \
model       scenario region variable                       unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2          dimensionless FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CH4 ppb           Cicero-SCM                    808.249   
                                                                         Cicero-SCM-ECS3               808.249   
                                                                         FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CO2 ppm           Cicero-SCM                    284.317   

time 

<IPython.core.display.Javascript object>

In [29]:
db["climatemodel"].unique()

array(['FaIR-1.5-DEFAULT', 'Cicero-SCM', 'Cicero-SCM-ECS3'], dtype=object)

<IPython.core.display.Javascript object>

### Minor quick fixes

We relabel all the ssp370-lowNTCF data to remove ambiguity.

In [30]:
db = db.timeseries().reset_index()
db["scenario"] = db["scenario"].apply(
    lambda x: "ssp370-lowNTCF-gidden" if x == "ssp370-lowNTCF" else x
)
db["scenario"] = db["scenario"].apply(
    lambda x: "esm-ssp370-lowNTCF-gidden" if x == "esm-ssp370-lowNTCF" else x
)
db["scenario"] = db["scenario"].apply(
    lambda x: "esm-ssp370-lowNTCF-gidden-allGHG"
    if x == "esm-ssp370-lowNTCF-allGHG"
    else x
)
db = ScmDataFrame(db)

<IPython.core.display.Javascript object>

In [31]:
assert "ssp370-lowNTCF" not in db["scenario"].unique().tolist()
assert "esm-ssp370-lowNTCF" not in db["scenario"].unique().tolist()
assert "esm-ssp370-lowNTCF-allGHG" not in db["scenario"].unique().tolist()

<IPython.core.display.Javascript object>

The Hector and MCE data is mislabelled so we do a quick fix here. I also have changed my mind about how to format the quantiles so tweak the FaIR and WASP data too. TODO: email modelling groups so they can fix it for phase 2.

In [32]:
mce_prob_data = db.filter(climatemodel="MCE*PROB*")
mce_prob_data["climatemodel"].unique()
if not mce_prob_data.timeseries().empty:
    mce_prob_data = mce_prob_data.timeseries().reset_index()

    def mce_get_quantile(inp):
        if inp.endswith("33rd"):
            return "33"

        if inp.endswith("67th"):
            return "67"

        raise NotImplementedError

    mce_prob_data["variable"] = (
        mce_prob_data["variable"]
        + "|"
        + mce_prob_data["climatemodel"].apply(mce_get_quantile)
        + "th quantile"
    )

    mce_prob_data["climatemodel"] = mce_prob_data["climatemodel"].apply(
        lambda x: "-".join(x.split("-")[:-1])
    )

    db = db.filter(climatemodel="MCE*PROB*", keep=False).append(mce_prob_data)

db.filter(climatemodel="MCE*PROB").head(10)

Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!


,,,,,time
model,scenario,region,variable,unit,climatemodel


<IPython.core.display.Javascript object>

In [33]:
hector_prob_data = db.filter(climatemodel="hector*HISTCALIB*")
if not hector_prob_data.timeseries().empty:
    hector_prob_data = hector_prob_data.timeseries().reset_index()

    def hector_get_quantile(inp):
        if inp.endswith("SD"):
            return "stddev"
        if inp.endswith("Mean"):
            return "mean"

        tmp = inp.split("q")[1]
        if len(tmp) == 3:
            tmp = "{}.{}".format(tmp[:2], tmp[2])
        if tmp.startswith("0"):
            tmp = tmp[1:]
        return tmp + "th quantile"

    hector_prob_data["variable"] = (
        hector_prob_data["variable"]
        + "|"
        + hector_prob_data["climatemodel"].apply(hector_get_quantile)
    )

    hector_prob_data["climatemodel"] = hector_prob_data["climatemodel"].apply(
        lambda x: x.split("-")[0]
    )

    db = db.filter(climatemodel="hector*HISTCALIB*", keep=False).append(
        hector_prob_data
    )

db.filter(climatemodel="hector*HISTCALIB").head(10)

Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!


,,,,,time
model,scenario,region,variable,unit,climatemodel


<IPython.core.display.Javascript object>

In [34]:
fair_prob_data = db.filter(climatemodel="*FaIR*")
if not fair_prob_data.timeseries().empty:
    fair_prob_data = fair_prob_data.timeseries().reset_index()

    fair_prob_data["variable"] = fair_prob_data["variable"].apply(
        lambda x: x.replace("|00th", "|0th").replace("|05th", "|5th")
    )

    db = db.filter(climatemodel="*FaIR*", keep=False).append(
        ScmDataFrame(fair_prob_data)
    )

db.filter(climatemodel="*FaIR*").head(10)

time                                                                                                        1850-01-01 00:00:00  \
model       scenario region variable                                        unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2                           dimensionless FaIR-1.5-DEFAULT                  NaN   
                            Atmospheric Concentrations|CH4                  ppb           FaIR-1.5-DEFAULT           808.249029   
                            Atmospheric Concentrations|CO2                  ppm           FaIR-1.5-DEFAULT           284.316999   
                            Atmospheric Concentrations|F-Gases|HFC|HFC125   ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC134a  ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC143a  ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC227ea ppt           FaIR-1.5-DEFAULT             0.000004   
                            Atmospheric Concentrations|F-Gases|HFC|HFC23    ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC245fa ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC32    ppt           FaIR-1.5-DEFAULT             0.000218   

time                                                                                                        1851-01-01 00:00:00  \
model       scenario region variable                                        unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2                           dimensionless FaIR-1.5-DEFAULT             1.000000   
                            Atmospheric Concentrations|CH4                  ppb           FaIR-1.5-DEFAULT           808.249029   
                            Atmospheric Concentrations|CO2                  ppm           FaIR-1.5-DEFAULT           287.160169   
                            Atmospheric Concentrations|F-Gases|HFC|HFC125   ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC134a  ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC143a  ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC227ea ppt           FaIR-1.5-DEFAULT             0.000004   
                            Atmospheric Concentrations|F-Gases|HFC|HFC23    ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC245fa ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|HFC32    ppt           FaIR-1.5-DEFAULT             0.000218   

time                                                                                                        1852-01-01 00:00:00  \
model       scenario region variable                                        unit          climatemodel                            
unspecified 1pctCO2  World  Airborne Fraction|CO2                           dimensionless FaIR-1.5-DEFAULT             0.886666   
                            Atmospheric Concentrations|CH4                  ppb           FaIR-1.5-DEFAULT           808.249029   
                            Atmospheric Concentrations|CO2                  ppm           FaIR-1.5-DEFAULT           290.031770   
                            Atmospheric Concentrations|F-Gases|HFC|HFC125   ppt           FaIR-1.5-DEFAULT             0.000000   
                            Atmospheric Concentrations|F-Gases|HFC|H

<IPython.core.display.Javascript object>

In [35]:
wasp_prob_data = db.filter(climatemodel="*WASP*")
if not wasp_prob_data.timeseries().empty:
    wasp_prob_data = wasp_prob_data.timeseries().reset_index()

    wasp_prob_data["variable"] = wasp_prob_data["variable"].apply(
        lambda x: x.replace("|00th", "|0th").replace("|05th", "|5th")
    )

    db = db.filter(climatemodel="*WASP*", keep=False).append(
        ScmDataFrame(wasp_prob_data)
    )

db.filter(climatemodel="*WASP*").head(10)

Filtered ScmDataFrame is empty!
Filtered ScmDataFrame is empty!


,,,,,time
model,scenario,region,variable,unit,climatemodel


<IPython.core.display.Javascript object>

## Unify units and check names

Here we loop over the submissions and unify their units as well as checking their naming matches what we expect.

In [36]:
base_df = db.timeseries()
any_failures = False

clean_db = []
for climatemodel, cdf in tqdm.tqdm_notebook(
    base_df.groupby("climatemodel"), desc="Climate model"
):
    print(climatemodel)
    print("-" * len(climatemodel))

    any_failures_climatemodel = False

    cdf = ScmDataFrame(cdf)
    cdf_converted_units = unify_units(cdf, protocol_variables)
    try:
        check_all_scenarios_as_in_protocol(cdf_converted_units, protocol_scenarios)
        check_all_variables_and_units_as_in_protocol(
            cdf_converted_units, protocol_variables
        )
    except AssertionError:
        any_failures_climatemodel = True
    #     # currently not possible as groups weren't told to obey variable hierarchy,
    #     # add this in phase 2
    #     for v_top in cdf_converted_units.filter(level=0)["variable"].unique():
    #         print(v_top)
    #         cdf_pyam = cdf_converted_units.filter(variable="{}*".format(v_top)).timeseries()
    #         cdf_pyam.columns = cdf_pyam.columns.map(lambda x: x.year)

    #         cdf_consistency_checker = pyam.IamDataFrame(cdf_pyam)
    #         if cdf_consistency_checker.check_internal_consistency() is not None:
    #             print("Failed for {}".format(v_top))
    #             any_failures_climatemodel = True
    #             failing_set = cdf_consistency_checker.copy()

    print()
    if not any_failures_climatemodel:
        clean_db.append(cdf_converted_units)
        print("All clear for {}".format(climatemodel))
    else:
        print("Failed {}".format(climatemodel))
        print("X" * len("Failed"))
        any_failures = True

    print()
    print()

if any_failures:
    raise AssertionError("database isn't ready yet")
else:
    clean_db = df_append(clean_db)
    clean_db.head()

Cicero-SCM
----------



All clear for Cicero-SCM


Cicero-SCM-ECS3
---------------



All clear for Cicero-SCM-ECS3


FaIR-1.5-DEFAULT
----------------


Failed for Effective Climate Feedback with target unit: W/m^2/K and current_unit: ['K/W/m^2' 'W/m^2/K']


set()
{'K/W/m^2'}

Failed FaIR-1.5-DEFAULT
XXXXXX





AssertionError: database isn't ready yet

<IPython.core.display.Javascript object>

In [ ]:
clean_db.head()

Notes whilst doing this:

- I wasn't clear that the variable hierarchy needs to be obeyed, hence doing internal consistency checks isn't going to work

For phase 2:

- checking internal consistency super slow, worth looping over top level variables when doing this to speed up filtering
- need to decide what a sensible tolerance is
- might have to go back to model notes to work out why there are inconsistencies
- will have to implement a custom hack to deal with the double counting in the direct aerosol forcing hierarchy

## Creating a database

In [ ]:
save_into_database(clean_db, OUTPUT_DATABASE_PATH, "rcmip-phase-1")